# VRPTW - M3AS

In [1]:
import numpy as np

In [2]:
class Cliente:
    def __init__(self, numero, x, y, demanda, inicio, fin, tiempo):
        self.numero = int(float(numero))
        self.x = float(x)
        self.y = float(y)
        self.demanda = float(demanda)
        self.inicio = float(inicio)
        self.fin = float(fin)
        self.tiempo = float(tiempo)

    @property
    def coords(self):
        return np.array([self.x, self.y])

In [3]:
# Preprocesar los datos de entrada
def procesar_instancia(filename):
    with open(filename, "r") as f:
        lineas = f.readlines()

    # Obtener clientes y capacidad
    clientes = int(lineas[1])
    capacidad = int(lineas[3])

    # Obtener por cada cliente los datos
    lista = []
    for cliente in lineas[5:]:
        lista.append(Cliente(*cliente.strip().split()))

    return capacidad, lista

In [7]:
capacidad_c101, clientes_c101 = procesar_instancia("datos/original/vrptw_c101.txt")
capacidad_rc101, clientes_rc101 = procesar_instancia("datos/original/vrptw_rc101.txt")

capacidad = capacidad_c101
clientes = clientes_c101
num_clientes = len(clientes)

In [54]:
def hallar_tiempo(i, j):
    return np.linalg.norm(clientes[i].coords - clientes[j].coords)

In [16]:
class Solucion:
    caminos = [[]]
    tiempo_actual = 0
    consumo_capacidad = 0
    
    def __init__(self):
        self.visitados = np.zeros(num_clientes, dtype=np.bool8)
        self.visitados[0] = True

    @property
    def num_vehiculos(self):
        return len(self.caminos)

    @property
    def nodo_actual(self):
        if len(self.caminos[-1]) > 0:
            _ , actual = self.caminos[-1][-1]
        else:
            actual = 0
        return actual

    def camino_en_vehiculo(self, i):
        return self.caminos[i]

    @property
    def completo(self):
        return False not in self.visitados

    def siguientes_nodos(self, i):
        no_visitados = np.where(self.visitados == False)
        siguientes_j = []

        for j in no_visitados:
            # Verificar que cumpla la ventana
            tiempo = hallar_tiempo(i, j)
            tiempo_al_llegar = max(self.tiempo_actual + tiempo, clientes[j].inicio)
            
            if tiempo_al_llegar + clientes[j].tiempo > clientes[j].fin:
                continue # No va a poder atenderle antes del cierre
            
            # Verificar que cumpla la capacidad del vehiculo
            if self.consumo_capacidad + clientes[j].demanda > capacidad:
                continue # Supera la capacidad

            # Se puede visitar
            siguientes_j.append(j)

        return siguientes_j

    def agregar_camino(self, j):
        # El sumamos al tiempo lo que cuesta llegar hasta j y su servicio (se incluye la espera si llega antes)
        self.tiempo_actual = max(hallar_tiempo(self.nodo_actual, j) + self.tiempo_actual, clientes[j].inicio) + clientes[j].tiempo
        self.visitados[j] = True
        self.consumo_capacidad = self.consumo_capacidad + clientes[j].demanda
        camino = (self.nodo_actual, j)
        self.caminos[-1].append( camino )

    def agregar_vehiculo(self):
        self.caminos.append([])
        self.tiempo_actual = 0

    def evaluar_tiempo_total(self):
        """
        Evalua el tiempo recorrido sin tener en cuenta el tiempo esperado
        """
        tiempo = 0.0
        for vehiculo in self.caminos:
            for viaje in vehiculo:
                tiempo = tiempo + hallar_tiempo(*viaje)

        return tiempo

In [56]:
m = 100
shape = (num_clientes, num_clientes)
vistado = np.zeros(shape, dtype=np.bool8)
tabla_t = np.empty(shape) # T_i,j
tabla_n_1 = np.empty(shape) # N_i,j
tabla_n_2 = np.empty(shape) # N_i,j
lambda_p = 0.1
beta = 0.5

In [58]:
# Inicializar visibilidad de tiempo
for i in range(num_clientes):
    for j in range(num_clientes):
        tabla_n_2[i,j] = 1.0 / hallar_tiempo(i,j) if hallar_tiempo(i,j) > 0 else 1

In [9]:
def feromona_calculada(i, j):
    visibilidades = ( tabla_n_1[i, j] ** lambda_p ) * (tabla_n_2[i, j] ** (1.0 - lambda_p))
    return tabla_t[i,j] * (visibilidades ** beta )

$$
p_{i,j} =
\begin{cases} 
      \frac{\tau_{i,j} \left[ n_{i,j}^0 \right]^{\lambda \beta} \left[ n_{i,j}^1 \right]^{(1-\lambda) \beta}}
      {\sum_{x \in J_i} \tau_{i,x} \left[ n_{i,x}^0 \right]^{\lambda \beta} \left[ n_{i,x}^1 \right]^{(1-\lambda) \beta}} & \text{si } j \in J_i \\
     0 & \text{ en caso contrario}
\end{cases}
$$

In [14]:
def probabilidades_transicion(i: int, nodos: [int]):
    probabilidades = np.array([ feromona_calculada(i,j) for j in nodos ])
    total_probabilidades = probabilidades.sum()
    return probabilidades / total_probabilidades

In [15]:
def seleccionar_siguiente_estado(solucion: Solucion):
    # Seleccionar siguiente nodo con distribución de probabilidad
    nodos_factibles = solucion.siguientes_nodos

    if len(nodos_factibles) > 0: # Hay nodos
        return np.random.choice(nodos_factibles, p=probabilidades_transicion(i, nodos_factibles))
    else:
        # Ya es muy tarde o ya atendió su capacidad máxima
        return 0 # Volvemos al deposito

In [17]:
def construir_solucion():
    solucion = Solucion()
    while not solucion.completo: # While existen estados no visitados
        siguiente = seleccionar_siguiente_estado()
        solucion.append(siguiente)
    return solucion

In [ ]:
generacion = 0
while True: # While not condición de parada
    generacion = generacion + 1
    for i in range(0, m): # Por cada hormiga
        # Construir solución
        # Evaluar solución
        # Actualizar Feromonas
        # Calcular mejor solución
        pass